In [3]:
!nvidia-smi

Wed Aug 13 22:46:43 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-40GB          Off |   00000000:00:04.0 Off |                    0 |
| N/A   31C    P0             43W /  400W |       0MiB /  40960MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [118]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("teamincribo/cyber-security-attacks")

print("Path to dataset files:", path)

Path to dataset files: /kaggle/input/cyber-security-attacks


In [132]:
import pandas as pd
import seaborn as sns
pathDF = "/kaggle/input/cyber-security-attacks/cybersecurity_attacks.csv"

cyberData = pd.read_csv(pathDF)
cyberData


,Timestamp,Source IP Address,Destination IP Address,Source Port,Destination Port,Protocol,Packet Length,Packet Type,Traffic Type,Payload Data,Malware Indicators,Anomaly Scores,Alerts/Warnings,Attack Type,Attack Signature,Action Taken,Severity Level,User Information,Device Information,Network Segment,Geo-location Data,Proxy Information,Firewall Logs,IDS/IPS Alerts,Log Source
0,2023-05-30 06:33:58,103.216.15.12,84.9.164.252,31225,17616,ICMP,503,Data,HTTP,Qui natus odio asperiores nam. Optio nobis ius...,IoC Detected,28.67,NaN,Malware,Known Pattern B,Logged,Low,Reyansh Dugal,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment A,"Jamshedpur, Sikkim",150.9.97.135,Log Data,NaN,Server
1,2020-08-26 07:08:30,78.199.217.198,66.191.137.154,17245,48166,ICMP,1174,Data,HTTP,Aperiam quos modi officiis veritatis rem. Omni...,IoC Detected,51.50,NaN,Malware,Known Pattern A,Blocked,Low,Sumer Rana,Mozilla/5.0 (compatible; MSIE 8.0; Windows NT ...,Segment B,"Bilaspur, Nagaland",NaN,Log Data,NaN,Firewall
2,2022-11-13 08:23:25,63.79.210.48,198.219.82.17,16811,53600,UDP,306,Control,HTTP,Perferendis sapiente vitae soluta. Hic delectu...,IoC Detected,87.42,Alert Triggered,DDoS,Known Pattern B,Ignored,Low,Himmat Karpe,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,Segment C,"Bokaro, Rajasthan",114.133.48.179,Log Data,Alert Data,Firewall
3,2023-07-02 10:38:46,163.42.196.10,101.228.192.255,20018,32534,UDP,385,Data,HTTP,Totam maxime beatae expedita explicabo porro l...,NaN,15.79,Alert Triggered,Malware,Known Pattern B,Blocked,Medium,Fateh Kibe,Mozilla/5.0 (Macintosh; PPC Mac OS X 10_11_5; ...,Segment B,"Jaunpur, Rajasthan",NaN,NaN,Alert Data,Firewall
4,2023-07-16 13:11:07,71.166.185.76,189.243.174.238,6131,26646,TCP,1462,Data,DNS,Odit nesciunt dolorem nisi iste iusto. Animi v...,NaN,0.52,Alert Triggered,DDoS,Known Pattern B,Blocked,Low,Dhanush Chad,Mozilla/5.0 (compatible; MSIE 5.0; Windows NT ...,Segment C,"Anantapur, Tripura",149.6.110.119,NaN,Alert Data,Firewall
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39995,2023-05-26 14:08:42,26.36.109.26,121.100.75.240,31005,6764,UDP,1428,Control,HTTP,Quibusdam ullam consequatur consequuntur accus...,IoC Detected,39.28,NaN,DDoS,Known Pattern A,Logged,Medium,Adira Madan,Mozilla/5.0 (iPad; CPU iPad OS 14_2_1 like Mac...,Segment A,"Nashik, Manipur",NaN,Log Data,Alert Data,Firewall
39996,2023-03-27 00:38:27,17.21.163.81,196.108.134.78,2553,28091,UDP,1184,Control,HTTP,Quaerat neque esse. Animi expedita natus commo...,IoC Detected,27.25,NaN,DDoS,Known Pattern A,Logged,High,Rati Dara,Mozilla/5.0 (Windows; U; Windows 98; Win 9x 4....,Segment C,"Vadodara, Mizoram",60.51.30.46,Log Data,NaN,Firewall
39997,2022-03-31 01:45:49,162.35.217.57,98.107.0.15,22505,25152,UDP,1043,Data,DNS,Enim at aspernatur illum. Saepe numquam eligen...,IoC Detected,31.01,NaN,DDoS,Known Pattern B,Blocked,Low,Samiha Joshi,Mozilla/5.0 (Windows; U; Windows NT 4.0) Apple...,Segment C,"Mahbubnagar, Himachal Pradesh",NaN,Log Data,Alert Data,Server
39998,2023-09-22 18:32:38,208.72.233.205,173.79.112.252,20013,2703,UDP,483,Data,FTP,Officiis dolorem sed harum provident earum dis...,IoC Detected,97.85,Alert Triggered,Malware,Known Pattern B,Ignored,Low,Rasha Chauhan,Mozilla/5.0 (X11; Linux i686) AppleWebKit/536....,Segment B,"Rourkela, Arunachal Pradesh",137.76.130.8,Log Data,NaN,Server


In [133]:
cyberData.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40000 entries, 0 to 39999
Data columns (total 25 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Timestamp               40000 non-null  object 
 1   Source IP Address       40000 non-null  object 
 2   Destination IP Address  40000 non-null  object 
 3   Source Port             40000 non-null  int64  
 4   Destination Port        40000 non-null  int64  
 5   Protocol                40000 non-null  object 
 6   Packet Length           40000 non-null  int64  
 7   Packet Type             40000 non-null  object 
 8   Traffic Type            40000 non-null  object 
 9   Payload Data            40000 non-null  object 
 10  Malware Indicators      20000 non-null  object 
 11  Anomaly Scores          40000 non-null  float64
 12  Alerts/Warnings         19933 non-null  object 
 13  Attack Type             40000 non-null  object 
 14  Attack Signature        40000 non-null

In [134]:
cyberData.corr(numeric_only=True)

,Source Port,Destination Port,Packet Length,Anomaly Scores
Source Port,1.000000,-0.005216,0.003657,0.004826
Destination Port,-0.005216,1.000000,0.002581,-0.003616
Packet Length,0.003657,0.002581,1.000000,-0.003599
Anomaly Scores,0.004826,-0.003616,-0.003599,1.000000


In [135]:
for col in cyberData.columns:
    print('\t%s: %d' % (col,cyberData[col].isna().sum()))

	Timestamp: 0
	Source IP Address: 0
	Destination IP Address: 0
	Source Port: 0
	Destination Port: 0
	Protocol: 0
	Packet Length: 0
	Packet Type: 0
	Traffic Type: 0
	Payload Data: 0
	Malware Indicators: 20000
	Anomaly Scores: 0
	Alerts/Warnings: 20067
	Attack Type: 0
	Attack Signature: 0
	Action Taken: 0
	Severity Level: 0
	User Information: 0
	Device Information: 0
	Network Segment: 0
	Geo-location Data: 0
	Proxy Information: 19851
	Firewall Logs: 19961
	IDS/IPS Alerts: 20050
	Log Source: 0


In [136]:
dupes = cyberData.duplicated()
print(dupes.sum())

0


In [137]:
print(cyberDataCleaned.dtypes)

Timestamp                  object
Source IP Address          object
Destination IP Address     object
Source Port                 int64
Destination Port            int64
Protocol                   object
Packet Length               int64
Packet Type                object
Traffic Type               object
Payload Data               object
Malware Indicators         object
Anomaly Scores            float64
Alerts/Warnings            object
Attack Type                object
Attack Signature           object
Action Taken               object
Severity Level             object
User Information           object
Device Information         object
Network Segment            object
Geo-location Data          object
Proxy Information          object
Firewall Logs              object
IDS/IPS Alerts             object
Log Source                 object
dtype: object


In [138]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import FeatureHasher
from sklearn.impute import SimpleImputer
import pandas as pd

# Updated feature lists based on your actual dataset
high_cardinality_features = ['Source IP Address', 'Destination IP Address']

low_cardinality_features = [
    'Protocol', 'Packet Type', 'Traffic Type', 'Attack Type',
    'Attack Signature', 'Action Taken', 'Severity Level',
    'User Information', 'Device Information', 'Network Segment',
    'Geo-location Data', 'Log Source'
]

# Categorical features with missing values that need imputation
categorical_with_missing = [
    'Malware Indicators', 'Alerts/Warnings', 'Proxy Information',
    'Firewall Logs', 'IDS/IPS Alerts'
]

numeric_features = ['Source Port', 'Destination Port', 'Packet Length', 'Anomaly Scores']

class ListWrapperTransformer:
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        # X is expected to be a DataFrame with multiple columns
        # Iterate over rows and then over values in each row
        return [[str(val) for val in row] for index, row in X.iterrows()]

    def fit_transform(self, X, y=None):
        return self.transform(X)

# Pipeline for low cardinality categorical features (no missing values)
onehot_pipeline = Pipeline([
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline for high cardinality features (no missing values expected)
hash_pipeline = Pipeline([
    ('wrap_list', ListWrapperTransformer()),
    ('hasher', FeatureHasher(n_features=32, input_type='string'))
])

# Pipeline for categorical features WITH missing values
categorical_missing_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),  # Fill with most frequent value
    ('onehot_encoder', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])

# Pipeline for numeric features
numeric_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),  # Fill numeric missing values with median
    ('scaler', StandardScaler())
])

# Create the main preprocessor
preprocessor = ColumnTransformer([
    ('cat_low', onehot_pipeline, low_cardinality_features),
    ('cat_high', hash_pipeline, high_cardinality_features),
    ('cat_missing', categorical_missing_pipeline, categorical_with_missing),
    ('num', numeric_pipeline, numeric_features)
], remainder='drop')  # Drop any remaining columns not specified

# Alternative: If you want to keep 'Timestamp' and 'Payload Data' columns,
# you might want to add them to appropriate feature lists or handle them separately

print("Feature categories:")
print(f"Low cardinality categorical: {len(low_cardinality_features)} features")
print(f"High cardinality categorical: {len(high_cardinality_features)} features")
print(f"Categorical with missing values: {len(categorical_with_missing)} features")
print(f"Numeric: {len(numeric_features)} features")
print(f"Total features to be processed: {len(low_cardinality_features) + len(high_cardinality_features) + len(categorical_with_missing) + len(numeric_features)}")

Feature categories:
Low cardinality categorical: 12 features
High cardinality categorical: 2 features
Categorical with missing values: 5 features
Numeric: 4 features
Total features to be processed: 23


In [141]:
cyberDataCleaned['Attack Type'].value_counts()

,count
Attack Type,
Intrusion,419
Malware,414
DDoS,404


In [140]:
from sklearn.model_selection import train_test_split
X = cyberDataCleaned.drop('Attack Type', axis = 1)
y = cyberDataCleaned['Attack Type']
X_train,X_test,y_train,y_test = train_test_split(X,y, test_size=.2, random_state=42, stratify=y)

X_train_encoded = preprocessor.fit_transform(X_train)
X_test_encoded = preprocessor.transform(X_test)


ValueError: A given column is not a column of the dataframe

In [102]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_test_encoded = label_encoder.transform(y_test)


In [103]:
print("TARGET VARIABLE ANALYSIS")
print("Attack Type value counts:")
attack_counts = cyberDataCleaned['Attack Type'].value_counts()
print(attack_counts)
print(f"\nNumber of unique attack types: {cyberDataCleaned['Attack Type'].nunique()}")



TARGET VARIABLE ANALYSIS
Attack Type value counts:
Attack Type
Phishing                                            539
Brute Force                                         529
Ping of Death                                       523
File Transfer Protocol (FTP) Attack                 517
Malware                                             504
Man-in-the-Middle (MITM)                            502
Simple Network Management Protocol (SNMP) Attack    497
Cross-Site Scripting (XSS) Attack                   496
DNS Tunneling                                       489
Advanced Persistent Threat (APT)                    489
Watering Hole Attack                                488
Email Spam                                          486
Zero-Day Exploit                                    486
Remote Desktop Protocol (RDP) Attack                484
SQL Injection                                       479
Insider Threat                                      476
Drive-by Download                        

In [104]:
print('Class Imbalance Analysis')
min_class_size = attack_counts.min()
max_class_size = attack_counts.max()
imbalance_ratio = max_class_size/min_class_size
print(f"Imbalance Ratio: {imbalance_ratio}")


Class Imbalance Analysis
Imbalance Ratio: 1.2085201793721974


In [105]:
print(X_train_encoded.shape)
print(X_test_encoded.shape)

(7860, 77)
(1966, 77)


In [20]:
#from sklearn.tree import DecisionTreeClassifier
#from sklearn.svm import SVC
#from sklearn.linear_model import SGDClassifier, RidgeClassifier
#param_grid = {
 #  'kernel': ['rbf'],
 #   'C': [0.1, 1, 10],
 #   'gamma': ['auto'],
 #   'class_weight': ['balanced']
#}
#grid_search = GridSearchCV(SVC(), param_grid, cv = 3)
#grid_search.fit(X_train_encoded, y_train_encoded)
#print(f"Best SVC model is: {grid_search.best_estimator_}")

In [106]:
from sklearn.svm import SVC
svc = SVC(kernel = 'poly', degree = 5, C = 1, class_weight='balanced')
svc.fit(X_train_encoded, y_train_encoded)
y_pred_svc = svc.predict(X_test_encoded)


In [107]:
from sklearn.metrics import accuracy_score
svc_accruacy = accuracy_score(y_test_encoded, y_pred_svc)
print(svc_accruacy)

0.04323499491353001


In [108]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import SGDClassifier, RidgeClassifier
sgd_clf = SGDClassifier(random_state= 42, early_stopping= True, eta0= .01, class_weight='balanced', penalty= 'l1')
sgd_clf.fit(X_train_encoded, y_train_encoded)
y_pred_sgd = sgd_clf.predict(X_test_encoded)
sgd_clf_acc = accuracy_score(y_test_encoded, y_pred_sgd)
print(sgd_clf_acc)


0.05239064089521872


In [109]:
ridge_clf = RidgeClassifier(alpha=1.0, class_weight='balanced')
ridge_clf.fit(X_train_encoded, y_train_encoded)
y_pred_ridge = ridge_clf.predict(X_test_encoded)
ridge_clf_acc = accuracy_score(y_test_encoded, y_pred_ridge)
print(ridge_clf_acc)

0.048830111902339775


In [110]:
dt_clf = DecisionTreeClassifier(class_weight='balanced', min_samples_split= 5)
dt_clf.fit(X_train_encoded, y_train_encoded)
y_pred_dt = dt_clf.predict(X_test_encoded)
dt_clf_acc = accuracy_score(y_test_encoded, y_pred_dt)
print(dt_clf_acc)

0.04374364191251272


In [115]:
import tensorflow as tf
from tensorflow import keras

nn = keras.Sequential([
    keras.layers.Dense(128, activation="relu", kernel_regularizer= tf.keras.regularizers.l2(0.02)), # Corrected regularizer
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(20, activation="softmax")
])
callback = keras.callbacks.EarlyStopping(
    monitor='val_loss',
    patience=3,
    restore_best_weights=True
)
nn.compile(loss="sparse_categorical_crossentropy", optimizer = keras.optimizers.Adam(learning_rate=1e-4)
, metrics=['accuracy'])
nn.summary()

Model: "sequential_16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_59 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_30 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_60 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_42          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_31 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_61 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_43          │ ?                      │   0 (unbuilt) │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_32 (Dropout)            │ ?                      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_62 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 0 (0.00 B)

In [116]:
nn.fit(X_train_encoded, y_train_encoded, epochs=30, verbose=1, validation_split= .2, callbacks= [callback])

Epoch 1/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 10s 24ms/step - accuracy: 0.0458 - loss: 5.5711 - val_accuracy: 0.0483 - val_loss: 4.9913
Epoch 2/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0523 - loss: 5.3757 - val_accuracy: 0.0464 - val_loss: 4.9257
Epoch 3/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0509 - loss: 5.2742 - val_accuracy: 0.0566 - val_loss: 4.8374
Epoch 4/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0519 - loss: 5.1392 - val_accuracy: 0.0566 - val_loss: 4.7469
Epoch 5/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0575 - loss: 4.9858 - val_accuracy: 0.0515 - val_loss: 4.6584
Epoch 6/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0537 - loss: 4.8840 - val_accuracy: 0.0534 - val_loss: 4.5749
Epoch 7/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0520 - loss: 4.7781 - val_accuracy: 0.0566 - val_loss: 4.4956
Epoch 8/30
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.0579 - loss: 4.6638 - val_accuracy: 

In [61]:
test_loss, test_accuracy = nn.evaluate(X_test_encoded, y_test_encoded)
print(f"\nTest accuracy: {test_accuracy:.4f}")

62/62 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.0443 - loss: 5.0503

Test accuracy: 0.0565


In [57]:
# 1. Simple preprocessing - just encode categoricals and scale numerics
simple_preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=False), nominal_categorical_features),
    ('num', StandardScaler(), numerical_features)
])

# 2. Prepare raw data without PCA or feature selection
X_raw = cyberDataCleaned.drop(['Attack Type'], axis=1)
y_raw = cyberDataCleaned['Attack Type']

# 3. Split the data
X_train_raw, X_test_raw, y_train_raw, y_test_raw = train_test_split(
    X_raw, y_raw, test_size=0.2, random_state=42, stratify=y_raw
)

# 4. Apply minimal preprocessing
X_train_simple = simple_preprocessor.fit_transform(X_train_raw)
X_test_simple = simple_preprocessor.transform(X_test_raw)

# 5. Encode target
y_train_simple = le.fit_transform(y_train_raw)
y_test_simple = le.transform(y_test_raw)

# 6. Train neural network on minimally processed data
nn_simple = keras.Sequential([
    keras.layers.Dense(128, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(64, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(32, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(20, activation="softmax")
])

nn_simple.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=0.001),
    metrics=['accuracy']
)

# 7. Fit and evaluate
history = nn_simple.fit(
    X_train_simple,
    y_train_simple,
    epochs=50,
    validation_split=0.2,
    verbose=1
)

# 8. Print evaluation metrics
test_loss, test_accuracy = nn_simple.evaluate(X_test_simple, y_test_simple)
print(f"\nTest accuracy with minimal preprocessing: {test_accuracy:.4f}")

Epoch 1/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 10s 25ms/step - accuracy: 0.0499 - loss: 3.4065 - val_accuracy: 0.0426 - val_loss: 3.0285
Epoch 2/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.1568 - loss: 2.7512 - val_accuracy: 0.0369 - val_loss: 3.0705
Epoch 3/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.3143 - loss: 2.3274 - val_accuracy: 0.0420 - val_loss: 3.1783
Epoch 4/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.4889 - loss: 1.8131 - val_accuracy: 0.0439 - val_loss: 3.3483
Epoch 5/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.6762 - loss: 1.2037 - val_accuracy: 0.0534 - val_loss: 3.5011
Epoch 6/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8194 - loss: 0.7138 - val_accuracy: 0.0560 - val_loss: 3.6429
Epoch 7/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8943 - loss: 0.4309 - val_accuracy: 0.0407 - val_loss: 3.8994
Epoch 8/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.9247 - loss: 0.3074 - val_accuracy: 